# QCoDeS Example with Tektronix Keithley 7510 Multimeter

In this example we will show how to use a few basic functions of the Keithley 7510 DMM. We attached the 1k Ohm resistor to the front terminals, with no source current or voltage.

For more detail about the 7510 DMM, please see the User's Manual: https://www.tek.com/digital-multimeter/high-resolution-digital-multimeters-manual/model-dmm7510-75-digit-graphical-sam-0, or Reference Manual: https://www.tek.com/digital-multimeter/high-resolution-digital-multimeters-manual-9

In [1]:
from qcodes.instrument_drivers.tektronix.keithley_7510 import Keithley7510

In [2]:
dmm = Keithley7510("dmm_7510", 'USB0::0x05E6::0x7510::04450363::INSTR')

Connected to: KEITHLEY INSTRUMENTS DMM7510 (serial:04450363, firmware:1.6.7d) in 0.07s


## To reset the system to default settings:

In [3]:
dmm.reset()

## To perform measurement with different sense functions:

When first turned on, the default sense function is for DC voltage

In [4]:
dmm.sense.function()

'voltage'

to perform the measurement:

In [5]:
dmm.sense.voltage()

-3.910893e-06

There'll be an error if try to call other functions, such as current:

In [6]:
try:
    dmm.sense.current()
except AttributeError as err:
    print(err)

'Sense7510' object and its delegates have no attribute 'current'


To switch between functions, do the following:

In [7]:
dmm.sense.function('current')
dmm.sense.function()

'current'

In [8]:
dmm.sense.current()

1.054986e-11

And of course, once the sense function is changed to 'current', the user can't make voltage measurement

In [9]:
try:
    dmm.sense.voltage()
except AttributeError as err:
    print(err)

'Sense7510' object and its delegates have no attribute 'voltage'


The available functions in the driver now are 'voltage', 'current', 'Avoltage', 'Acurrent', 'resistance', and 'Fresistance', where 'A' means 'AC', and 'F' means 'Four-wire'

In [10]:
try:
    dmm.sense.function('ac current')
except ValueError as err:
    print(err)

("'ac current' is not in {'current', 'Acurrent', 'resistance', 'Fresistance', 'Avoltage', 'voltage'}; Parameter: dmm_7510.sense_function", 'setting dmm_7510_sense_function to ac current')


## To set measurement range (positive full-scale measure range):

By default, the auto range is on

In [11]:
dmm.sense.auto_range()

True

We can change it to 'off' as following

In [12]:
dmm.sense.auto_range(0)
dmm.sense.auto_range()

False

Note: this auto range setting is for the sense function at this moment, which is 'current'

In [13]:
dmm.sense.function()

'current'

If switch to another function, the auto range is still on, by default

In [14]:
dmm.sense.function('voltage')
dmm.sense.function()

'voltage'

In [15]:
dmm.sense.auto_range()

True

to change the range, use the following

In [16]:
dmm.sense.range(10)

In [17]:
dmm.sense.range()

10.0

This will also automatically turn off the auto range:

In [18]:
dmm.sense.auto_range()

False

the allowed range (upper limit) value is a set of discrete numbers, for example, 100mV, 1V, 10V, 100V, 100V. If a value other than those allowed values is input, the system will just use the "closest" one:

In [19]:
dmm.sense.range(150)
dmm.sense.range()

1000.0

In [20]:
dmm.sense.range(105)
dmm.sense.range()

100.0

The driver will not give any error messages for the example above, but if the value is too large or too small, there'll be an error message:

In [21]:
try:
    dmm.sense.range(0.0001)
except ValueError as err:
    print(err)

('0.0001 is invalid: must be between 0.1 and 1000 inclusive; Parameter: dmm_7510_sense.range', 'setting dmm_7510_sense_range to 0.0001')


## To set the NPLC (Number of Power Line Cycles) value for measurements:

By default, the NPLC is 1 for each sense function

In [22]:
dmm.sense.nplc()

1.0

To set the NPLC value:

In [23]:
dmm.sense.nplc(.1)
dmm.sense.nplc()

0.1

Same as the 'range' variable, each sense function has its own NPLC value:

In [24]:
dmm.sense.function('resistance')
dmm.sense.function()

'resistance'

In [25]:
dmm.sense.nplc()

1.0

## To set the delay:

By default, the auto delay is enabled. According to the guide, "When this is enabled, a delay is added after a range or function change to allow the instrument to settle." But it's unclear how much the delay is.

In [26]:
dmm.sense.auto_delay()

True

To turn off the auto delay:

In [27]:
dmm.sense.auto_delay(0)
dmm.sense.auto_delay()

False

To turn the auto delay back on:

In [28]:
dmm.sense.auto_delay(1)
dmm.sense.auto_delay()

True

There is also an "user_delay", but it is designed for rigger model, please see the user guide for detail.

To set the user delay time:

First to set a user number to relate the delay time with: (default user number is empty, so an user number has to be set before setting the delay time)

In [29]:
dmm.sense.user_number(1)
dmm.sense.user_number()

1

By default, the user delay is 0s:

In [30]:
dmm.sense.user_delay()

'0'

Then to set the user delay as following:

In [31]:
dmm.sense.user_delay(0.1)
dmm.sense.user_delay()

'0.1'

The user delay is tied to user number:

In [32]:
dmm.sense.user_number(2)
dmm.sense.user_number()

2

In [33]:
dmm.sense.user_delay()

'0'

For the record, the auto delay here is still on:

In [34]:
dmm.sense.auto_delay()

True

## To set auto zero (automatic updates to the internal reference measurements):

By default, the auto zero is on

In [35]:
dmm.sense.auto_zero()

True

To turn off auto zero:

In [36]:
dmm.sense.auto_zero(0)
dmm.sense.auto_zero()

False

The auto zero setting is also tied to each function, not universal:

In [37]:
dmm.sense.function('current')
dmm.sense.function()

'current'

In [38]:
dmm.sense.auto_zero()

True

There is way to ask the system to do auto zero once:

In [39]:
dmm.sense.auto_zero_once()

See P487 of the Reference Manual for how to use auto zero ONCE. Note: it's not funtion-dependent.

## To set averaging filter for measurements, including average count, and filter type:

By default, averaging is off:

In [40]:
dmm.sense.average()

False

To turn it on:

In [41]:
dmm.sense.average(1)
dmm.sense.average()

True

Default average count value is 10, **remember to turn average on**, or it will not work:

In [42]:
dmm.sense.average_count()

'10'

To change the average count:

In [43]:
dmm.sense.average_count(23)
dmm.sense.average_count()

'23'

The range for average count is 1 to 100:

In [44]:
try:
    dmm.sense.average_count(200)
except ValueError as err:
    print(err)

('200 is invalid: must be between 1 and 100 inclusive; Parameter: dmm_7510_sense.average_count', 'setting dmm_7510_sense_average_count to 200')


There are two average types, repeating (default) or moving filter:

In [45]:
dmm.sense.average_type()

'REP'

To make changes:

In [46]:
dmm.sense.average_type('MOV')
dmm.sense.average_type()

'MOV'